Load data with Pandas from a CSV

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|', low_memory=False)
print(f"Number of rows in file: {df.shape[0]}")

Number of rows in file: 6187847


Load data with Pandas from a Parquet

In [2]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.shape[0]}")

Number of rows in file: 6187847


Test PySpark from parquet

In [3]:
import os
import sys

# Set Hadoop environment variables
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'

# Create SparkSession with datetime rebase configuration
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("LocalSpark") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
    .getOrCreate()

df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.count()}")

Number of rows in file: 6187847


Dask from CSV

In [4]:
import dask.dataframe as dd

df = dd.read_csv(
    r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv",
    sep='|',
    dtype = 'object',
    low_memory=False,  
)
print(f"Number of rows: {df.shape[0].compute()}")

c:\Users\es422\anaconda3\envs\geo\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 12.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Number of rows: 6187847


Dask from parquet

In [5]:
import dask.dataframe as dd

df = dd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows: {df.shape[0].compute()}")

Number of rows: 6187847
